In [102]:
from __future__ import print_function
import numpy as np
import numpy as np
import random
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

In [103]:
def train_scaler(X):
    scaler = MaxAbsScaler()
    scaler.fit(X)
    return scaler    

In [104]:
def load_training_data(csv_file_name, undersample=False):
    """Loads a CSV file produced by the Java feature generator."""
    X_raw = []
    y_raw = []
    random.seed(21)
    with open(csv_file_name, 'r') as f:
        for line_number, line in enumerate(f.readlines()):
            parts = line[:-1].split(",")
            if undersample:
                if int(parts[0])==0:
                    if random.random() < 0.15:
                        y_raw.append(int(parts[0]))
                        features_np = np.array([float(f) for f in parts[1:]])
                        X_raw.append(features_np) 
                else:
                    y_raw.append(int(parts[0]))
                    features_np = np.array([float(f) for f in parts[1:]])
                    X_raw.append(features_np)  
            else:
                y_raw.append(int(parts[0]))
                features_np = np.array([float(f) for f in parts[1:]])
                X_raw.append(features_np)                    
        X_raw = np.array(X_raw)
        y_raw = np.array(y_raw)

        print("Feature shape: {0}".format(X_raw.shape))
        print("Label shape: {0}".format(y_raw.shape))
        return np.nan_to_num(X_raw), np.nan_to_num(y_raw)

In [105]:
csv_file_name_train = "../../../../data/nonempty_train.csv"
csv_file_name_test = "../../../../data/nonempty_valid.csv"

In [106]:
X_train, y_train = load_training_data(csv_file_name_train,undersample=True)
X_test, y_test = load_training_data(csv_file_name_test)

Feature shape: (118, 5)
Label shape: (118,)
Feature shape: (249, 5)
Label shape: (249,)


In [107]:
scaler = train_scaler(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [108]:
train_pos = np.sum(y_train == 1)
test_pos = np.sum(y_test == 1)
print("We have {0} positive labels in training set.".format(train_pos))
print("We have {0} positive labels in validation set.".format(test_pos))

We have 51 positive labels in training set.
We have 28 positive labels in validation set.


In [109]:
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [110]:
clf = SGDClassifier(alpha=0.0002, loss='log', penalty='l2', n_iter=10000)
#clf = LinearDiscriminantAnalysis()
clf.fit(X_train,y_train)
th = 0.89

y_prob = clf.predict_proba(X_test)
y_predict = np.zeros(len(y_test))
for i in range(len(y_test)):
    prediction = 0
    if y_prob[i,1] >= th:
        y_predict[i] = 1
        
        
fp = fn = tp = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i] == 1:
        tp += 1
    if y_predict[i] == 1 and y_test[i] == 0:
        fp += 1
    if y_predict[i] == 0 and y_test[i] == 1:
        fn += 1
print("True empty predictions {0}!".format(tp))
print("{0} were predicted to be empty but were not; {1} predicted to be nonempty but were not;".format(fp,fn))

True empty predictions 9!
7 were predicted to be empty but were not; 19 predicted to be nonempty but were not;
